In [1]:
import numpy as np
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1558344694965_0003,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark Text Encoder example") \
    .getOrCreate()

VBox()

In [3]:
rev_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
revs = spark.read.csv(rev_data,header=True,sep='\t')

VBox()

In [4]:
revs.printSchema()

VBox()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [4]:
#the total number of reviews
total_num = revs.count()
total_num

VBox()

4751577

In [6]:
#the number of unique users
num_unique_users = revs.select('customer_id').distinct().count()
num_unique_users

VBox()

1940732

In [9]:
#the number of unique products
num_unique_products = revs.select('product_id').distinct().count()
num_unique_products 

VBox()

782326

In [7]:
#the largest number of reviews published by a single user
largest_revs_num = revs.groupby('customer_id').count().orderBy('count', ascending=False).limit(1).collect()
largest_revs_num

VBox()

[Row(customer_id='50736950', count=7168)]

In [16]:
#the top 10 users ranked by the number of reviews they publish
top_10_users = revs.groupby('customer_id').count().orderBy('count', ascending=False).limit(10).show()
top_10_users

VBox()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5412|
|   51184997| 5369|
|   18116317| 4222|
|   23267387| 4023|
|   50345651| 3793|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+

In [12]:
#the median number of reviews published by a user
median_revs_of_user = revs.groupby('customer_id').count().approxQuantile('count',[0.5],0.05)
median_revs_of_user

VBox()

[1.0]

In [9]:
#the largest number of reviews written for a single product
largest_revs_product = revs.groupby('product_id').count().orderBy('count', ascending=False).limit(1).collect()
largest_revs_product

VBox()

[Row(product_id=u'B00008OWZG', count=3936)]

In [11]:
#the top 10 products ranked by the number of reviews they have
top_10_products = revs.groupby('product_id').count().orderBy('count', ascending=False).limit(10).show()
top_10_products

VBox()

+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
|B0000AGWEC| 3326|
|B00MIA0KGY| 2699|
|B00NEJ7MMI| 2420|
|B000089RVX| 2376|
|B004EBT5CU| 2106|
|B0026P3G12| 2080|
|B00009PRZF| 2026|
|B00004XONN| 1901|
|B00006J6VG| 1793|
+----------+-----+

In [8]:
#the median number of reviews a product has
median_revs_of_product = revs.groupby('product_id').count().approxQuantile('count',[0.5],0.05)
median_revs_of_product

VBox()

[2.0]